In [1]:
import os
import json
from pathlib import Path
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.documentintelligence import DocumentIntelligenceClient

In [ ]:


# Option A: Use environment variables (recommended)
endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT", "https://az-t1.cognitiveservices.azure.com/")
key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY", "")


if not endpoint or not key:
    raise ValueError("Missing Azure endpoint/key. Set env vars or fill manually above.")

client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

# Supported models
SUPPORTED_MODELS = {
    "layout": "prebuilt-layout",
    "read": "prebuilt-read",
    "invoice": "prebuilt-invoice",
}


In [4]:
# 📌 Cell 4: Helper functions

def analyze_with_url(model_id: str, url: str):
    poller = client.begin_analyze_document(model_id=model_id, body={"urlSource": url})
    return poller.result()

def analyze_with_file(model_id: str, file_path: str):
    p = Path(file_path).expanduser()
    if not p.exists():
        raise FileNotFoundError(f"File not found: {p}")
    poller = client.begin_analyze_document(model_id=model_id, body=p.read_bytes())
    return poller.result()

def print_summary(model_key: str, result):
    print(f"\n=== Model: {model_key} ({SUPPORTED_MODELS[model_key]}) ===")
    print(f"API version: {result.get('apiVersion', 'unknown')}")
    print(f"Pages analyzed: {len(result.get('pages', []))}")
    
    content = result.get("content")
    if content:
        preview = content[:400].replace("\n", " ")
        print("\n-- Text Preview --")
        print(preview + ("..." if len(content) > 400 else ""))
    
    if model_key == "invoice":
        docs = result.get("documents") or []
        if docs:
            fields = docs[0].get("fields", {})
            print("\n-- Invoice Fields --")
            for key, val in fields.items():
                print(f"{key:<20}: {val.get('content')} (conf={val.get('confidence')})")

def save_result(result, out_file="result.json"):
    Path(out_file).write_text(json.dumps(result, indent=2, ensure_ascii=False))
    print(f"\n✅ Saved raw JSON to {out_file}")


In [ ]:
model_choice = "layout"  
model_id = SUPPORTED_MODELS[model_choice]

# Option A: Local file
file_path = "ChatGPT Image Aug 22, 2025, 02_48_08 PM.png"   # change this
try:
    result = analyze_with_file(model_id, file_path)
    print_summary(model_choice, result)
except HttpResponseError as e:
    print("API Error:", e)


=== Model: layout (prebuilt-layout) ===
API version: 2024-11-30
Pages analyzed: 1

-- Text Preview --
₹ Invoice Receipt ACME SUPPLIES LTD. 123 Elm St. Anytown, USA 12345 INVOICE Invoice No. INV-1001 Invoice Date March 15 2024 Bill To: John Doe 456 Oak Ave. Sometown, USA 67890 Description Quantity Unit Price Line Total Pens 10 $1.50 $15,00 Paper 5 $6,00 $30,00 Subtotal $45,00 Tax 8,00% Total $48,60


TypeError: Object of type AnalyzeResult is not JSON serializable

In [6]:
result

{'apiVersion': '2024-11-30', 'modelId': 'prebuilt-layout', 'stringIndexType': 'textElements', 'content': '₹\nInvoice Receipt\nACME SUPPLIES LTD. 123 Elm St. Anytown, USA 12345\nINVOICE\nInvoice No. INV-1001\nInvoice Date March 15 2024\nBill To:\nJohn Doe\n456 Oak Ave.\nSometown, USA 67890\nDescription\nQuantity\nUnit Price\nLine Total\nPens\n10\n$1.50\n$15,00\nPaper\n5\n$6,00\n$30,00\nSubtotal\n$45,00\nTax\n8,00%\nTotal\n$48,60', 'pages': [{'pageNumber': 1, 'angle': 0.05408459156751633, 'width': 1024, 'height': 1536, 'unit': 'pixel', 'words': [{'content': '₹', 'polygon': [144, 265, 182, 266, 182, 306, 144, 306], 'confidence': 0.164, 'span': {'offset': 0, 'length': 1}}, {'content': 'Invoice', 'polygon': [303, 257, 505, 256, 505, 317, 302, 314], 'confidence': 0.992, 'span': {'offset': 2, 'length': 7}}, {'content': 'Receipt', 'polygon': [527, 256, 742, 257, 742, 319, 527, 317], 'confidence': 0.995, 'span': {'offset': 10, 'length': 7}}, {'content': 'ACME', 'polygon': [180, 446, 269, 446, 2

In [7]:
model_choice = "read"  
model_id = SUPPORTED_MODELS[model_choice]

# Option A: Local file
file_path = "ChatGPT Image Aug 22, 2025, 02_48_08 PM.png"   # change this
try:
    result = analyze_with_file(model_id, file_path)
    print_summary(model_choice, result)
except HttpResponseError as e:
    print("API Error:", e)


=== Model: read (prebuilt-read) ===
API version: 2024-11-30
Pages analyzed: 1

-- Text Preview --
K Invoice Receipt ACME SUPPLIES LTD. 123 Elm St. Anytown, USA 12345 INVOICE Invoice No. INV-1001 Invoice Date March 15 2024 Bill To: John Doe 456 Oak Ave. Sometown, USA 67890 Description Quantity Unit Price Line Total Pens 10 $1.50 $15,00 Paper 5 $6,00 $30,00 Subtotal $45,00 Tax 8,00% Total $48,60


In [8]:
model_choice = "invoice"  
model_id = SUPPORTED_MODELS[model_choice]

# Option A: Local file
file_path = "ChatGPT Image Aug 22, 2025, 02_48_08 PM.png"   # change this
try:
    result = analyze_with_file(model_id, file_path)
    print_summary(model_choice, result)
except HttpResponseError as e:
    print("API Error:", e)


=== Model: invoice (prebuilt-invoice) ===
API version: 2024-11-30
Pages analyzed: 1

-- Text Preview --
₹ Invoice Receipt ACME SUPPLIES LTD. INVOICE 123 Elm St. Anytown, USA 12345 Invoice No. INV-1001 Invoice Date March 15 Bill To: 2024 John Doe 456 Oak Ave. Sometown, USA 67890 Description Quantity Unit Price Line Total Pens 10 $1.50 $15,00 Paper 5 $6,00 $30,00 Subtotal $45,00 Tax 8,00% Total $48,60

-- Invoice Fields --
BillingAddress      : 456 Oak Ave.
Sometown, USA 67890 (conf=0.918)
BillingAddressRecipient: John Doe (conf=0.948)
CustomerName        : John Doe (conf=0.948)
InvoiceDate         : March 15
2024 (conf=0.577)
InvoiceId           : INV-1001 (conf=0.988)
InvoiceTotal        : $48,60 (conf=0.987)
Items               : None (conf=None)
SubTotal            : $45,00 (conf=0.987)
VendorAddress       : 123 Elm St.
Anytown, USA 12345 (conf=0.918)
VendorAddressRecipient: ACME SUPPLIES LTD. (conf=0.942)
VendorName          : ACME SUPPLIES LTD. (conf=0.942)


In [9]:
client = DocumentIntelligenceClient(endpoint, AzureKeyCredential(key))

In [16]:
with open("ChatGPT Image Aug 22, 2025, 02_48_08 PM.png", "rb") as f:
    poller = client.begin_analyze_document(
        model_id="prebuilt-layout",
        body=f.read(),
        output_content_format="markdown",  # get Markdown in result["content"]
    )

In [17]:
result = poller.result()
print("Pages:", len(result.get("pages", [])))
print("First 600 chars of Markdown:\n", (result.get("content") or ""))

Pages: 1
First 600 chars of Markdown:
 ₹


# Invoice Receipt

ACME SUPPLIES LTD.
123 Elm St.
Anytown, USA 12345


## INVOICE

Invoice No. INV-1001

Invoice Date March 15
2024

Bill To:

John Doe

456 Oak Ave.

Sometown, USA 67890


<table>
<tr>
<th>Description</th>
<th>Quantity</th>
<th>Unit Price</th>
<th>Line Total</th>
</tr>
<tr>
<td>Pens</td>
<td>10</td>
<td>$1.50</td>
<td>$15,00</td>
</tr>
<tr>
<td>Paper</td>
<td>5</td>
<td>$6,00</td>
<td>$30,00</td>
</tr>
</table>


<table>
<tr>
<td>Subtotal</td>
<td>$45,00</td>
</tr>
<tr>
<td>Tax</td>
<td>8,00%</td>
</tr>
<tr>
<td>Total</td>
<td>$48,60</td>
</tr>
</table>



In [18]:
with open("CCStatement_Current21-08-2025.pdf", "rb") as f:
    poller = client.begin_analyze_document(
        model_id="prebuilt-layout",
        body=f.read(),
        output_content_format="markdown",  # get Markdown in result["content"]
    )

In [25]:
result = poller.result()
print("Pages:", len(result.get("pages", [])))
print(result.get("content"))
with open("CCStatement_Current21-08-2025.html", "w") as f:
    f.write(result.get("content") or "")

Pages: 2
<figure>

ICICI Bank

</figure>


# VIEW CURRENT STATEMENT

My Credit Card Details for
AMIYA MANDAL-037474 ****** 0009


<table>
<tr>
<th>Card Holder Name</th>
<th>AMIYA MANDAL</th>
<th>Card Type</th>
<th>primary</th>
</tr>
<tr>
<td>Available Credit Limit</td>
<td>INR 0</td>
<td>Total Credit Limit</td>
<td>INR 0</td>
</tr>
<tr>
<td>Available Cash Limit</td>
<td>INR 0</td>
<td>Total Cash Limit</td>
<td>INR 0</td>
</tr>
<tr>
<td>Total Amount Due</td>
<td>INR 0</td>
<td>Purchases and Other Charges</td>
<td>INR 364044.59</td>
</tr>
<tr>
<td>Cash Advances</td>
<td>INR 0</td>
<td>Payments and Other Credits</td>
<td>INR 309086.16</td>
</tr>
<tr>
<td>Statement Date</td>
<td>21-08-2025</td>
<td>Previous Balance</td>
<td>INR 48150.7</td>
</tr>
</table>


Statement Period

29-07-2025 TO 21-08-2025

My Reward point details


<table>
<tr>
<th>Opening Balance</th>
<th>Earned During the Month</th>
<th>Redeemed During Month</th>
<th>Closing Balance</th>
</tr>
<tr>
<td>0</td>
<td>0</td>
<td>0<

In [23]:
result

{'apiVersion': '2024-11-30', 'modelId': 'prebuilt-layout', 'stringIndexType': 'textElements', 'content': '<figure>\n\nICICI Bank\n\n</figure>\n\n\n# VIEW CURRENT STATEMENT\n\nMy Credit Card Details for\nAMIYA MANDAL-037474 ****** 0009\n\n\n<table>\n<tr>\n<th>Card Holder Name</th>\n<th>AMIYA MANDAL</th>\n<th>Card Type</th>\n<th>primary</th>\n</tr>\n<tr>\n<td>Available Credit Limit</td>\n<td>INR 0</td>\n<td>Total Credit Limit</td>\n<td>INR 0</td>\n</tr>\n<tr>\n<td>Available Cash Limit</td>\n<td>INR 0</td>\n<td>Total Cash Limit</td>\n<td>INR 0</td>\n</tr>\n<tr>\n<td>Total Amount Due</td>\n<td>INR 0</td>\n<td>Purchases and Other Charges</td>\n<td>INR 364044.59</td>\n</tr>\n<tr>\n<td>Cash Advances</td>\n<td>INR 0</td>\n<td>Payments and Other Credits</td>\n<td>INR 309086.16</td>\n</tr>\n<tr>\n<td>Statement Date</td>\n<td>21-08-2025</td>\n<td>Previous Balance</td>\n<td>INR 48150.7</td>\n</tr>\n</table>\n\n\nStatement Period\n\n29-07-2025 TO 21-08-2025\n\nMy Reward point details\n\n\n<table>\n


📘 Azure Document Intelligence — Feature & Cost Overview

1. Service Coverage & Capabilities

Azure Document Intelligence provides three main categories of models for document automation:
	•	Document Analysis Models
	•	Extract text (printed & handwritten), tables, and key–value pairs.
	•	General-purpose processing across various document types.
	•	Prebuilt Models
	•	Ready-to-use for invoices, receipts, IDs, business cards, contracts, US tax forms, etc.
	•	25+ specialized models for finance, legal, and government workflows.
	•	Saves training time; suitable for standard document formats.
	•	Custom Models
	•	Train for unique formats and complex documents.
	•	Types:
	•	Custom Neural Models → Mixed or semi-structured layouts
	•	Custom Template Models → Static formats
	•	Custom Classification Models → Detect doc type before extraction

⸻

2. Pricing Structure

Azure charges per page processed (per API call). Pricing (as of Aug 2025):

Tier / Model	Cost per 1,000 pages	Cost per Page	Notes
Free Tier	500 pages / month free	$0.00	Limited testing
Read Model	$1.50 (0–1M pages)	$0.0015	Drops to $0.0006/page after 1M pages
Prebuilt Models	$10.00	$0.010	Invoices, receipts, ID docs
Custom Classification	$3.00	$0.003	Detects doc type
Custom Extraction	$50.00	$0.050	Specialized data fields

🔹 Cost Optimization
	•	Commitment tiers: Save 20–30% (Custom Extraction drops from $50 → $35 per 1,000 pages).
	•	Caching: Prevents reprocessing charges.
	•	Model selection: Use cheaper Read Model when advanced features aren’t required.

⸻

3. Performance & Speed

Processing time varies by model, API version, and document complexity.

Model	Avg. Processing Time	Notes
Read Model	3–5 sec	Fastest option
Prebuilt Invoice	8–12 sec	Handles tables, totals
Custom Classification	7–10 sec	Type detection
Custom Extraction	12–25 sec	Complex but accurate

	•	New API (2024-02-29-preview): 3–6x faster (5–10 sec/doc).
	•	Older API (2023-10-31-preview): 30+ sec/doc.
	•	Tier Impact:
	•	Free (F0) → 1 call / 2 sec (rate limited)
	•	Standard (S0) → 15 calls/sec (~5 sec per doc)
	•	Premium (S1) → 20 calls/sec (~3 sec per doc)

⸻

4. Cost & Time Examples

Scenario	Cost	Time
Single PDF (1 page, Read)	$0.0015	3 sec
Invoice (3 pages, Prebuilt)	$0.030	15 sec
Custom Form (5 pages, Extraction)	$0.250	25 sec
Batch (100 docs, Read)	$0.150	300 sec


⸻

5. Python Implementation Guide
	•	Package: azure-ai-documentintelligence
	•	Setup: Python 3.8+, API Key + Endpoint (env variables recommended).
	•	Core Use Cases:
	•	Basic analysis (layout/text extraction)
	•	Prebuilt invoices/receipts
	•	Custom model training & inference
	•	Batch jobs for scalability
	•	Error handling with retries

Advanced Features:
	•	Barcode reading
	•	Formula recognition
	•	Searchable PDFs
	•	High-resolution image mode (extra cost)

⸻

6. Service Comparison

Service	Use Case	Cost	Notes
Document Intelligence	Structured data, tables, fields	$0.0015–$0.050/page	Most advanced
Computer Vision OCR	Simple text extraction	$5.00 / 1,000 calls	No table/key-value support
Text Analytics	NLP (sentiment, entities, key phrases)	Free 5K calls / month, then pay-per-call	Use with DI for pipelines

Integration: Use Document Intelligence for extraction → pass to Text Analytics for NLP.

⸻

7. Key Cost Optimization Strategies
	1.	Model Selection → Use Read Model for simple docs, Custom only for special cases.
	2.	Volume Discounts → Commit to monthly tier (up to 30% off).
	3.	Batch Processing → Group docs, avoid repeated calls.
	4.	API Updates → Use latest preview for speed boosts.
	5.	Caching Results → Store processed docs to avoid duplicate charges.
